# **TUGAS BESAR SI**

KELOMPOK 8

RB

Anggota :
1. Muhammad Zaki Abdillah
2. Halilah Roja Nasywa
3. Veni Zahara Kartika
4. Elsyah Sapyrah
5. Muhammad Kaisar Firdaus

## Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from collections import defaultdict
from numpy import inf
from itertools import groupby
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import pairwise_distances

## Data Movies and Rating

In [ ]:
# Data Rating
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/content/u.data', sep='\t', names=ratings_cols)

# Data Movie
movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/u.item', sep='|', names=movies_cols, usecols=range(5), encoding='latin-1')

# Data Genre
column_names = ['movie_id', 'movie_title', 'release_date', 'video_release_date',
                'imdb_url', 'unknown', 'Action', 'Adventure', 'Animation',
                'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                'Thriller', 'War', 'Western']
genre = pd.read_csv('/content/u.info', sep="\t", names=["info"])

# Data user
column_names = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
user = pd.read_csv('/content/u.user', sep="|", names=column_names)

In [ ]:
df = pd.merge(ratings, movies)
df = df[['user_id', 'movie_id', 'rating', 'title']]
df

,user_id,movie_id,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)
...,...,...,...,...
99995,840,1674,4,Mamma Roma (1962)
99996,655,1640,3,"Eighth Day, The (1996)"
99997,655,1637,3,Girls Town (1996)
99998,655,1630,3,"Silence of the Palace, The (Saimt el Qusur) (1..."


## Matriks Rating

In [ ]:
# Utility Matrix
ratings_matrix = df.pivot_table(index=['user_id'],
                 columns=['movie_id'], values='rating').reset_index(drop=True)

ratings_matrix.fillna(0, inplace=True)
ratings_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
939,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Matriks Rating dengan Pearson Correlation

In [ ]:
movie_similarities = np.corrcoef(ratings_matrix)
item_similarity_matrix = pd.DataFrame(movie_similarities)
item_similarity_matrix = 1 - item_similarity_matrix
item_similarity_matrix = pd.DataFrame(item_similarity_matrix)
item_similarity_matrix.shape
item_similarity_matrix

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,0.000000,0.893678,1.016424,0.978284,6.974079e-01,0.654024,0.679983,0.723792,0.961188,0.711509,...,0.714077,0.931122,0.810728,0.853021,0.880556,9.398247e-01,7.525168e-01,0.884496,0.891016,6.767692e-01
1,0.893678,0.000000,0.917320,0.839677,9.797822e-01,0.802156,0.977114,0.927228,0.856284,0.893139,...,0.893584,0.711450,0.674767,0.591534,0.710888,7.959056e-01,8.091519e-01,0.855823,0.861708,9.457647e-01
2,1.016424,0.917320,0.000000,0.668326,1.028322e+00,0.982327,1.013384,0.944615,0.956495,0.987920,...,1.020605,0.981254,0.875826,0.953753,0.911026,1.001145e+00,8.724611e-01,0.915468,0.897975,1.023601e+00
3,0.978284,0.839677,0.668326,0.000000,1.002426e+00,0.969794,0.958944,0.828884,0.910628,0.975982,...,0.983359,0.980262,0.894865,0.820402,0.877351,9.891096e-01,8.250961e-01,0.859189,0.850425,9.746325e-01
4,0.697408,0.979782,1.028322,1.002426,1.110223e-16,0.846591,0.720323,0.790720,0.974504,0.882834,...,0.727134,0.960667,0.986001,0.960481,0.914452,9.757076e-01,8.175337e-01,0.887429,0.903397,7.510087e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.939825,0.795906,1.001145,0.989110,9.757076e-01,0.940513,0.967409,0.931730,0.978331,0.981843,...,0.984815,0.583221,0.776827,0.792682,0.589530,1.110223e-16,9.504829e-01,0.835261,0.991964,8.992515e-01
939,0.752517,0.809152,0.872461,0.825096,8.175337e-01,0.704051,0.748012,0.786592,0.903114,0.712417,...,0.726922,0.925995,0.871776,0.847778,0.875059,9.504829e-01,1.110223e-16,0.877522,0.778766,8.173868e-01
940,0.884496,0.855823,0.915468,0.859189,8.874288e-01,0.886142,0.991832,0.870904,0.867600,0.942818,...,0.987113,0.810034,0.730630,0.778304,0.702812,8.352606e-01,8.775225e-01,0.000000,0.919870,9.347004e-01
941,0.891016,0.861708,0.897975,0.850425,9.033972e-01,0.733655,0.789610,0.857168,0.929305,0.845705,...,0.827632,0.956761,0.968500,0.898052,0.948568,9.919641e-01,7.787662e-01,0.919870,0.000000,8.722646e-01


In [ ]:
# Export matrix item to csv
item_similarity_matrix.to_csv('matrix_item_pearson.csv', index=False)

## Pencarian Similarity User dengan FireFly

In [ ]:
def calculate_time(distance, speed):
    return 0 if speed == 0 else distance/speed

def get_distance(data, row, col):
    return data.iloc[row-1][col-1]

def calculate_total_distance(data, route):
    total_distance = 0
    for i in range(len(route)-1):
        distance = get_distance(data, route[i], route[i+1])
        if distance == 0:
            return inf  # No edge found
        total_distance += distance
        if route[i+1] == num_items:
            return total_distance  # End Route
    return total_distance

def print_status(status_route, route, time):
    print(status_route, route, '\nTime: ', time, '\n\n')

def initialize():
    data = item_similarity_matrix
    main_route = []
    main_route.extend(range(0, num_items))
    main_route_distance = calculate_total_distance(data, main_route)
    main_route_speed = 1  # delay between userId
    return data

def calculate_light_intensity(data, speed, route):
    distance = calculate_total_distance(data, route)
    if distance == inf:
        return 0
    time = calculate_time(distance, speed)
    return 0 if time == 0 else 1/time

def get_light_intensity_from_all_fireflies(population, data, speed):
    lights = []
    for route in population:
        lights.append(calculate_light_intensity(data, speed, route))
    return lights

def initialize_population(num_population, num_dimensions):
    population = []

    for i in range(num_population):
        firefly = []
        firefly.extend(np.random.choice(range(2, num_dimensions+1), num_dimensions-1, replace=False))  # Random permutation, without replacement
        population.append(firefly)
    return population

def order_population(light, population):
    indices = np.argsort(light, axis=0)  # sorting ascending
    light_ = [light[idx] for idx in indices]
    population_ = [population[idx] for idx in indices]
    return light_, population_

def evaluate_population(population):
    return [k for k, v in groupby(sorted(population))]

In [ ]:
num_items = 943
np.seterr(divide='ignore', invalid='ignore')
data = initialize()
alternative_route_speed = 1  # m/min = 20km/h
gamma = 0.01  # Absorption coefficient
beta0 = 1  # Attractiveness constant
alpha = 1.0  # Randomness strength 0--1 (highly random)
theta = 0.97  # Randomness reduction factor theta=10^(-5/tMax)
num_population = 943
num_dimensions = 943  # Dimension

lb = 0
ub = 943
scale = ub - lb

best_firefly = []
solution_found = False
population = initialize_population(num_population, num_dimensions)
temp_population = []

light = get_light_intensity_from_all_fireflies(population, data, alternative_route_speed)

light_, population_ = order_population(light, population)

# best firefly check
if light_[len(light_)-1] > 0:
    solution_found = True
    best_firefly = population_[len(population_)-1]

while not solution_found:
    alpha *= theta
    population.extend(temp_population)
    population = evaluate_population(population)  # Remove duplicate population
    light = get_light_intensity_from_all_fireflies(population, data, alternative_route_speed)
    light_, population_ = order_population(light, population)

    # best firefly check
    if light_[len(light_)-1] > 0:
        solution_found = True
        best_firefly = population_[len(population_)-1]
        break

    temp_population = []

    # Movement/mutation of firefly population
    for i in range(len(population_)):
        for j in range(i):
            if light_[j] >= light_[i]:
                selected_firefly = population_[i]
                r = np.sqrt(np.sum([k**2 for k in ([a-b for a, b in zip(population_[i], population_[j])])]))  # Euclidean distance
                beta = beta0 * math.exp(-gamma * (r**2))
                index_for_swap = int(scale * np.random.uniform(0, 1, 1) + lb)
                if alpha * np.random.uniform(0, 1, 1) > 0.5:
                    index2_for_swap = index_for_swap + int(beta) + lb
                else:
                    index2_for_swap = index_for_swap - int(beta) - lb
                if index2_for_swap > ub:
                    selected_firefly[index_for_swap-1], selected_firefly[ub-1] = selected_firefly[ub-1], selected_firefly[index_for_swap-1]
                elif index2_for_swap < lb:
                    selected_firefly[index_for_swap-1], selected_firefly[lb-1] = selected_firefly[lb-1], selected_firefly[index_for_swap-1]
                else:
                    selected_firefly[index_for_swap-1], selected_firefly[index2_for_swap-1] = selected_firefly[index2_for_swap-1], selected_firefly[index_for_swap-1]
                temp_population.append(selected_firefly)

print('Best Firefly: ', best_firefly)

Best Firefly:  [195, 630, 943, 656, 117, 804, 756, 940, 864, 613, 45, 663, 819, 488, 811, 214, 313, 606, 569, 62, 402, 915, 369, 636, 672, 730, 283, 750, 206, 221, 132, 312, 99, 705, 897, 388, 516, 813, 827, 869, 871, 121, 854, 522, 900, 793, 14, 668, 433, 145, 523, 358, 469, 832, 473, 941, 781, 410, 938, 80, 416, 191, 579, 144, 650, 379, 39, 506, 578, 507, 840, 449, 361, 398, 445, 789, 266, 196, 497, 554, 475, 170, 243, 285, 56, 458, 153, 9, 306, 888, 29, 525, 73, 722, 618, 34, 367, 98, 249, 338, 305, 230, 40, 430, 681, 893, 593, 732, 889, 744, 232, 624, 602, 118, 485, 184, 812, 308, 319, 166, 89, 453, 318, 894, 805, 711, 514, 922, 718, 400, 483, 324, 275, 70, 521, 929, 148, 771, 942, 689, 149, 102, 3, 389, 908, 853, 344, 484, 109, 773, 343, 466, 240, 654, 653, 30, 737, 728, 621, 279, 382, 891, 217, 108, 107, 36, 394, 775, 231, 667, 808, 684, 403, 741, 861, 159, 359, 464, 434, 461, 442, 874, 256, 317, 350, 764, 489, 510, 251, 412, 133, 377, 205, 524, 661, 758, 619, 143, 180, 146, 331,

In [ ]:
bf = pd.DataFrame(best_firefly)
bf.rename(columns={0: 'user_id'}, inplace=True)
lg = pd.DataFrame(sorted(light_, reverse=True))
lg.rename(columns={0: 'weight'}, inplace=True)
result = pd.concat([bf, lg], axis=1)
result

,user_id,weight
0,195.0,0.585707
1,630.0,0.544848
2,943.0,0.507883
3,656.0,0.355367
4,117.0,0.301915
...,...,...
938,414.0,0.001230
939,576.0,0.001228
940,260.0,0.001228
941,311.0,0.001225


In [ ]:
result = result.dropna()
result

,user_id,weight
0,195.0,0.585707
1,630.0,0.544848
2,943.0,0.507883
3,656.0,0.355367
4,117.0,0.301915
...,...,...
937,482.0,0.001230
938,414.0,0.001230
939,576.0,0.001228
940,260.0,0.001228


In [ ]:
result.describe()

,user_id,weight
count,942.000000,942.000000
mean,472.500000,0.009588
std,272.076276,0.039956
min,2.000000,0.001225
25%,237.250000,0.001652
50%,472.500000,0.002530
75%,707.750000,0.004986
max,943.000000,0.585707


## Prediksi Rating User Based Collaborative Filtering

In [ ]:
def predict_rating(user_id, item_id, ratings_matrix, similarity_matrix):
    similarity_df = pd.DataFrame(similarity_matrix, index=ratings_matrix.index, columns=ratings_matrix.index)
    rated_users = ratings_matrix.iloc[:, item_id]
    similar_users = similarity_df[user_id]
    similar_rated_users = similar_users.dropna()
    rated_users = rated_users[similar_rated_users.index]
    weighted_sum = (rated_users * similar_rated_users).sum()
    sum_of_weights = similar_rated_users.abs().sum()
    if sum_of_weights != 0:
        prediction = weighted_sum / sum_of_weights
    else:
        prediction = 0
    return prediction

target_user_id = 566
target_item_id = 1
prediction = predict_rating(target_user_id, target_item_id, ratings_matrix, movie_similarities)
print("Prediksi rating untuk pengguna {} pada item {}: {:.2f}".format(target_user_id, target_item_id, prediction))

Prediksi rating untuk pengguna 566 pada item 1: 0.60


## Evaluasi Prediksi Rating dengan MAE dan MSE

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

predicted_rating = 0.60
actual_rating = 3
mae = mean_absolute_error([actual_rating], [predicted_rating])
mse = mean_squared_error([actual_rating], [predicted_rating])

print("MAE: {:.2f}".format(mae))
print("MSE: {:.2f}".format(mse))

MAE: 2.40
MSE: 5.76
